In [17]:
import polars

import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Explore Dataset

In [2]:
df = polars.read_csv("../data/raw/_dataset senior data scientist_ Tech Challenge - Sheet1.csv")
df.describe()

describe,USER_ID,REGISTRATION_AT,TOTAL_VISIT_COUNT,LAST_VISIT_AT,TOTAL_POST_COUNT,LAST_POST_AT,TOTAL_LIKES_RECEIVED,LAST_LIKE_RECEIVED_AT,TOTAL_COMMENTS_RECEIVED,LAST_COMMENT_RECEIVED_AT,TOTAL_LIKES_GIVEN,LAST_LIKE_GIVEN_AT,TOTAL_COMMENTS_WRITTEN,LAST_COMMENT_WRITTEN_AT,CHURNED
str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64
"""count""",50000.0,"""50000""",50000.0,"""50000""",50000.0,"""50000""",50000.0,"""50000""",50000.0,"""50000""",50000.0,"""50000""",50000.0,"""50000""",50000.0
"""null_count""",0.0,"""0""",0.0,"""0""",38078.0,"""38078""",38078.0,"""43607""",38078.0,"""41856""",36786.0,"""36786""",38339.0,"""38339""",0.0
"""mean""",25000.5,null,37.2042,null,2.36051,null,2.467623,null,4.843483,null,5.270622,null,4.151273,null,0.5
"""std""",14433.901067,null,59.466454,null,4.143234,null,8.533281,null,11.759609,null,16.896868,null,9.83948,null,0.500005
"""min""",1.0,"""2022-03-01T00:…",1.0,"""2022-01-02""",1.0,"""2022-03-01T13:…",0.0,"""2022-03-02T05:…",0.0,"""2022-03-01T22:…",1.0,"""2022-03-01T07:…",1.0,"""2022-03-01T09:…",0.0
"""max""",50000.0,"""2023-02-27T23:…",491.0,"""2023-07-20""",212.0,"""2023-07-19T21:…",384.0,"""2023-07-19T22:…",529.0,"""2023-07-19T22:…",720.0,"""2023-07-19T22:…",313.0,"""2023-07-19T22:…",1.0
"""median""",25000.5,null,13.0,null,1.0,null,1.0,null,2.0,null,2.0,null,2.0,null,0.5
"""25%""",12501.0,null,4.0,null,1.0,null,0.0,null,0.0,null,1.0,null,1.0,null,null
"""75%""",37501.0,null,42.0,null,2.0,null,2.0,null,5.0,null,4.0,null,4.0,null,null


In [3]:
df["USER_ID"].n_unique() == len(df)

True

Some observations:
- Dataset is nicely balanced with respect to the training target ``churned``
- Rest of the data is either a date or a numerical value, which should be easy to work with
- Does not contain duplicates with respect to `USER_ID`
- Dates are provided but we do not know about the "date of churning". Without expressing the times in a relative manner the model will learn potentially learn from the absolut times instead of learning from the time differences to the churning event, which will likely result in a bad generalization of the model in practice. It is probably safer to not consider these columns as features for the model in a raw fashion.

## Convert feature columns to numerical values

- All `TOTAL_*` columns are counts and contain lots of null values. A simple approach could be to fill them with `-1` as "lower than 0"
- Date can be converted to a timestamp to get an easy numerical value (may require some rescaling to be easier on the model)

In [4]:
df = polars.read_csv("../data/raw/_dataset senior data scientist_ Tech Challenge - Sheet1.csv").with_columns([
    polars.col("^*_AT$").str.to_datetime().dt.timestamp(),  # Convert all date columns to a timestamp
    polars.col("^TOTAL_.*$").fill_null(value=-1)  # Fill all count columns with zero if no data is available
])

In [5]:
df.describe()

describe,USER_ID,REGISTRATION_AT,TOTAL_VISIT_COUNT,LAST_VISIT_AT,TOTAL_POST_COUNT,LAST_POST_AT,TOTAL_LIKES_RECEIVED,LAST_LIKE_RECEIVED_AT,TOTAL_COMMENTS_RECEIVED,LAST_COMMENT_RECEIVED_AT,TOTAL_LIKES_GIVEN,LAST_LIKE_GIVEN_AT,TOTAL_COMMENTS_WRITTEN,LAST_COMMENT_WRITTEN_AT,CHURNED
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0,50000.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,38078.0,0.0,43607.0,0.0,41856.0,0.0,36786.0,0.0,38339.0,0.0
"""mean""",25000.5,1.6622e15,37.2042,1.6788e15,-0.19872,1.6721e15,-0.17318,1.6720e15,0.39332,1.6723e15,0.6572,1.6732e15,0.20138,1.6735e15,0.5
"""std""",14433.901067,8.6956e12,59.466454,1.1810e13,2.478627,1.1700e13,4.420957,1.1591e13,6.258754,1.1664e13,9.11561,1.1658e13,5.227161,1.1620e13,0.500005
"""min""",1.0,1.6461e15,1.0,1.6411e15,-1.0,1.6461e15,-1.0,1.6462e15,-1.0,1.6462e15,-1.0,1.6461e15,-1.0,1.6461e15,0.0
"""max""",50000.0,1.6775e15,491.0,1.6898e15,212.0,1.6898e15,384.0,1.6898e15,529.0,1.6898e15,720.0,1.6898e15,313.0,1.6898e15,1.0
"""median""",25000.5,1.6619e15,13.0,1.6845e15,-1.0,1.6734e15,-1.0,1.6733e15,-1.0,1.6736e15,-1.0,1.6745e15,-1.0,1.6750e15,0.5
"""25%""",12501.0,1.6553e15,4.0,1.6699e15,-1.0,1.6631e15,-1.0,1.6630e15,-1.0,1.6631e15,-1.0,1.6639e15,-1.0,1.6644e15,null
"""75%""",37501.0,1.6691e15,42.0,1.6889e15,-1.0,1.6824e15,-1.0,1.6821e15,-1.0,1.6826e15,1.0,1.6836e15,-1.0,1.6839e15,null


## Analyzing Intercorrelations

Checking for intercorrelations of the different columns:
- `LAST_VISIT_AT` is strongly negatively correlated to churning, as expected. The longer the user was absent the more clear it becomes that the has grown desinterested in the product.
- Same but less pronounced for `TOTAL_VISIT_COUNT`, which is basically reflecting the same thing.
- There is strong correlation between the many different "interactions", such as `TOTAL_COMMENTS_RECEIVED` and `TOTAL_POST_COUNT` essentially expressing: users that are active are posting a lot, getting answers, giving feedback, getting feedback and so on...

In [16]:
fig = px.imshow(
    df.corr(), 
    zmin=-1.0, 
    zmax=1.0,
    text_auto=True, 
    aspect="auto",
    y=df.columns
)
fig.show()

## Addressing the Datetime Columns Concern

The datetime columns are very problematic for generating a good model in the real world. In particular, if the model learns a connection along the line of "users whose last login was prior to data X are likely to churn" it is incapable to adapt to absolute times in future cases. In particular, for a model trained on data months old, data X might not seem to inidicate churning, while in reality the last login was long ago and churning is indeed likely. The best remedy would be to express the dates relative to when the churning was detected or concluded. This way we could train a model that predicts the probability of churning "today". However, since this is not possible I rather opt for droping this columns all together for the baseline model. A more sophisticated model may attempt to build sensible features from the time differences between, e.g., `REGISTRATION_AT` and `LAST_VISIT_AT` ...

## Simple Date-Agnostic Baseline Model

In [22]:
train_data = (
    df.select([
        polars.col("^TOTAL_.*$"), 
        polars.col("CHURNED")]
    )
    .with_columns([polars.col("CHURNED").cast(polars.Int8)])
    .rename(dict(CHURNED="label"))
)
train_df, test_df = train_test_split(train_data, test_size=0.2)
train_features = train_df.drop("label")
train_label = train_df["label"]
test_features = test_df.drop("label")
test_label = test_df["label"]

In [31]:
random_forest_clf = RandomForestClassifier()
random_forest_clf.fit(train_features, train_label)
print("TRAIN - Classification Report")
print(classification_report(train_label, random_forest_clf.predict(train_features)))
print("TEST - Classification Report")
print(classification_report(test_label, random_forest_clf.predict(test_features)))
print("FEATURE IMPORTANCE")
for feature, importance in zip(train_df.columns, random_forest_clf.feature_importances_):
    print(f"{feature:>30}: {importance}")

TRAIN - Classification Report
              precision    recall  f1-score   support

           0       0.85      0.81      0.83     19954
           1       0.82      0.86      0.84     20046

    accuracy                           0.83     40000
   macro avg       0.84      0.83      0.83     40000
weighted avg       0.84      0.83      0.83     40000

TEST - Classification Report
              precision    recall  f1-score   support

           0       0.79      0.77      0.78      5046
           1       0.77      0.79      0.78      4954

    accuracy                           0.78     10000
   macro avg       0.78      0.78      0.78     10000
weighted avg       0.78      0.78      0.78     10000

FEATURE IMPORTANCE
             TOTAL_VISIT_COUNT: 0.7245491548971829
              TOTAL_POST_COUNT: 0.026173787772303184
          TOTAL_LIKES_RECEIVED: 0.035239642170787136
       TOTAL_COMMENTS_RECEIVED: 0.04582754390385835
             TOTAL_LIKES_GIVEN: 0.08415966141607008
       

Already with a very simple model, we can achieve quite useful performance with about `~78%`  test F1 score. This is mostly leveraged by the `TOTAL_VISIT_COUNT`, basically identifying that users that visit frequently are less likely to churn from the platform, as they are invested.

Some important remarks:
- here would be the time when business decisions matter: What is the churn mitigation strategy? If it is cheap, we likely care about the `recall` as we want to attempt to prevent churn on all potential users that are prone to churning. However, if the prevention strategy is expensive, we may care more about the `precision` to ensure that the effort to prevent churning is going to the users that are most likely to churn.
- In fact, talking about prevention strategy, it might be more useful to identify users that are likely to change their mind if we engage in some prevention strategy. Essentially distinguishing churning users between "can be saved" and "beyond saving". 
- There is so much more to consider that it would take forever to spell out. If you are interested in my thoughts, feel free to contact me.